### Import necessary packages

In [116]:
import pandas as pd
import numpy as np
from nba_api.stats.static import players, teams
from nba_api.stats.endpoints import shotchartdetail
import matplotlib.pyplot as plt
from matplotlib.patches import Circle, Rectangle, Arc
from math import dist
import plotly.graph_objects as go

pd.set_option('display.max_columns', 500)

### Define function that will draw the basketball court

In [117]:
def draw_plotly_court(fig, fig_width=600, margins=10):
        
    # From: https://community.plot.ly/t/arc-shape-with-path/7205/5
    def ellipse_arc(x_center=0.0, y_center=0.0, a=10.5, b=10.5, start_angle=0.0, end_angle=2 * np.pi, N=200, closed=False):
        t = np.linspace(start_angle, end_angle, N)
        x = x_center + a * np.cos(t)
        y = y_center + b * np.sin(t)
        path = f'M {x[0]}, {y[0]}'
        for k in range(1, len(t)):
            path += f'L{x[k]}, {y[k]}'
        if closed:
            path += ' Z'
        return path

    fig_height = fig_width * (470 + 2 * margins) / (500 + 2 * margins)
    fig.update_layout(width=fig_width, height=fig_height)

    # Set axes ranges
    fig.update_xaxes(range=[-250 - margins, 250 + margins])
    fig.update_yaxes(range=[-52.5 - margins, 417.5 + margins])

    threept_break_y = 89.47765084
    three_line_col = "#777777"
    main_line_col = "#777777"

    fig.update_layout(
        # Line Horizontal
        margin=dict(l=20, r=20, t=20, b=20),
        paper_bgcolor="white",
        plot_bgcolor="white",
        yaxis=dict(
            scaleanchor="x",
            scaleratio=1,
            showgrid=False,
            zeroline=False,
            showline=False,
            ticks='',
            showticklabels=False,
            fixedrange=True,
        ),
        xaxis=dict(
            showgrid=False,
            zeroline=False,
            showline=False,
            ticks='',
            showticklabels=False,
            fixedrange=True,
        ),
        shapes=[
            dict(
                type="rect", x0=-250, y0=-52.5, x1=250, y1=417.5,
                line=dict(color=main_line_col, width=1),
                # fillcolor='#333333',
                layer='below'
            ),
            dict(
                type="rect", x0=-80, y0=-52.5, x1=80, y1=137.5,
                line=dict(color=main_line_col, width=1),
                # fillcolor='#333333',
                layer='below'
            ),
            dict(
                type="rect", x0=-60, y0=-52.5, x1=60, y1=137.5,
                line=dict(color=main_line_col, width=1),
                # fillcolor='#333333',
                layer='below'
            ),
            dict(
                type="circle", x0=-60, y0=77.5, x1=60, y1=197.5, xref="x", yref="y",
                line=dict(color=main_line_col, width=1),
                # fillcolor='#dddddd',
                layer='below'
            ),
            dict(
                type="line", x0=-60, y0=137.5, x1=60, y1=137.5,
                line=dict(color=main_line_col, width=1),
                layer='below'
            ),

            dict(
                type="rect", x0=-2, y0=-7.25, x1=2, y1=-12.5,
                line=dict(color="#ec7607", width=1),
                fillcolor='#ec7607',
            ),
            dict(
                type="circle", x0=-7.5, y0=-7.5, x1=7.5, y1=7.5, xref="x", yref="y",
                line=dict(color="#ec7607", width=1),
            ),
            dict(
                type="line", x0=-30, y0=-12.5, x1=30, y1=-12.5,
                line=dict(color="#ec7607", width=1),
            ),

            dict(type="path",
                 path=ellipse_arc(a=40, b=40, start_angle=0, end_angle=np.pi),
                 line=dict(color=main_line_col, width=1), layer='below'),
            dict(type="path",
                 path=ellipse_arc(a=237.5, b=237.5, start_angle=0.386283101, end_angle=np.pi - 0.386283101),
                 line=dict(color=main_line_col, width=1), layer='below'),
            dict(
                type="line", x0=-220, y0=-52.5, x1=-220, y1=threept_break_y,
                line=dict(color=three_line_col, width=1), layer='below'
            ),
            dict(
                type="line", x0=-220, y0=-52.5, x1=-220, y1=threept_break_y,
                line=dict(color=three_line_col, width=1), layer='below'
            ),
            dict(
                type="line", x0=220, y0=-52.5, x1=220, y1=threept_break_y,
                line=dict(color=three_line_col, width=1), layer='below'
            ),

            dict(
                type="line", x0=-250, y0=227.5, x1=-220, y1=227.5,
                line=dict(color=main_line_col, width=1), layer='below'
            ),
            dict(
                type="line", x0=250, y0=227.5, x1=220, y1=227.5,
                line=dict(color=main_line_col, width=1), layer='below'
            ),
            dict(
                type="line", x0=-90, y0=17.5, x1=-80, y1=17.5,
                line=dict(color=main_line_col, width=1), layer='below'
            ),
            dict(
                type="line", x0=-90, y0=27.5, x1=-80, y1=27.5,
                line=dict(color=main_line_col, width=1), layer='below'
            ),
            dict(
                type="line", x0=-90, y0=57.5, x1=-80, y1=57.5,
                line=dict(color=main_line_col, width=1), layer='below'
            ),
            dict(
                type="line", x0=-90, y0=87.5, x1=-80, y1=87.5,
                line=dict(color=main_line_col, width=1), layer='below'
            ),
            dict(
                type="line", x0=90, y0=17.5, x1=80, y1=17.5,
                line=dict(color=main_line_col, width=1), layer='below'
            ),
            dict(
                type="line", x0=90, y0=27.5, x1=80, y1=27.5,
                line=dict(color=main_line_col, width=1), layer='below'
            ),
            dict(
                type="line", x0=90, y0=57.5, x1=80, y1=57.5,
                line=dict(color=main_line_col, width=1), layer='below'
            ),
            dict(
                type="line", x0=90, y0=87.5, x1=80, y1=87.5,
                line=dict(color=main_line_col, width=1), layer='below'
            ),

            dict(type="path",
                 path=ellipse_arc(y_center=417.5, a=60, b=60, start_angle=-0, end_angle=-np.pi),
                 line=dict(color=main_line_col, width=1), layer='below'),

        ]
    )
    return True

### Download team data to dictionary

In [135]:
player_dict = players.get_active_players()
team_dict = teams.get_teams()

### Add the Celtics' shooting data to a data frame 

In [136]:
# Get data for all shots taken by Boston Celtics players this season
shotlog = shotchartdetail.ShotChartDetail(team_id = 1610612738,
    player_id=0,
    context_measure_simple = 'FGA',
    season_type_all_star = ['Regular Season'])

shot_df = shotlog.get_data_frames()[0]
shot_df.head()

,GRID_TYPE,GAME_ID,GAME_EVENT_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,PERIOD,MINUTES_REMAINING,SECONDS_REMAINING,EVENT_TYPE,ACTION_TYPE,SHOT_TYPE,SHOT_ZONE_BASIC,SHOT_ZONE_AREA,SHOT_ZONE_RANGE,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG,GAME_DATE,HTM,VTM
0,Shot Chart Detail,0022200001,11,203935,Marcus Smart,1610612738,Boston Celtics,1,11,15,Made Shot,Driving Floating Bank Jump Shot,2PT Field Goal,Mid-Range,Right Side(R),8-16 ft.,13,120,55,1,1,20221018,BOS,PHI
1,Shot Chart Detail,0022200001,15,1628369,Jayson Tatum,1610612738,Boston Celtics,1,10,46,Made Shot,Jump Shot,3PT Field Goal,Left Corner 3,Left Side(L),24+ ft.,23,-232,49,1,1,20221018,BOS,PHI
2,Shot Chart Detail,0022200001,29,1628401,Derrick White,1610612738,Boston Celtics,1,10,4,Missed Shot,Running Layup Shot,2PT Field Goal,Restricted Area,Center(C),Less Than 8 ft.,1,-10,12,1,0,20221018,BOS,PHI
3,Shot Chart Detail,0022200001,35,201143,Al Horford,1610612738,Boston Celtics,1,9,53,Missed Shot,Cutting Layup Shot,2PT Field Goal,Restricted Area,Center(C),Less Than 8 ft.,2,20,10,1,0,20221018,BOS,PHI
4,Shot Chart Detail,0022200001,39,1628369,Jayson Tatum,1610612738,Boston Celtics,1,9,42,Made Shot,Running Layup Shot,2PT Field Goal,Restricted Area,Center(C),Less Than 8 ft.,0,-2,-4,1,1,20221018,BOS,PHI


### Create Spatial Dataset

Each shot attempt and shot made is matched to its closest coordinate. The lists of made attemps and attempts are used to calculate accuracy at each coordinate.

In [134]:
from scipy.spatial import KDTree
# create lists of x / y coordinate values for each point
coords = list()

# iterate through x & y ranges
for i in range(-250, 250, 5):
    for j in range(-50, 430, 5):
        coords.append((i, j))

#Create list of coordinates for shot locations and for shot made flags
shot_coords = list(zip(shot_df.LOC_X.values, shot_df.LOC_Y.values))
made_flags = list(shot_df.SHOT_MADE_FLAG.values)

tree = KDTree(coords)

# Initialize shots take and made lists as lists of 0s that associate with each coordinate.
shots_taken = [0 for i in range(len(coords))]
shots_made = [0 for i in range(len(coords))]

# Map each shot to the closest coordinate
for shot, made in zip(shot_coords,made_flags):
    dist, index = tree.query(shot)
    shots_taken[index] += 1
    if made == 1:
        shots_made[index] += 1

# Separate coordinates into x/y lists for use in graphs.
x_locs = [x[0] for x in coords]
y_locs = [y[1] for y in coords]

# Calculate accuracy at each coordinate pair
shot_accuracy = [made/taken if taken > 0 else 0 for made, taken in zip(shots_made, shots_taken)]


### Create chart of all shots by Boston Celtics

In [130]:
fig = go.Figure()

draw_plotly_court(fig)

fig.add_trace(go.Scatter(
    x=x_locs, y=y_locs, mode='markers', name='markers',
    marker=dict(
        size=shots_taken, sizemode='area', sizeref=2. * max(shots_taken) / (11. ** 2), sizemin=2.5,
        color=shot_accuracy,
        line=dict(width=1, color='#333333'), symbol='square',
    ),
))
fig.show(config=dict(displayModeBar=False))

### Add accuracy colorscale

In [139]:
colorscale = 'Greens'
marker_cmin = 0.1
marker_cmax = 0.8
ticktexts = [str(marker_cmin*100)+'%-','',str(marker_cmax*100)+'%+']

fig = go.Figure()
draw_plotly_court(fig)
fig.add_trace(go.Scatter(
    x=x_locs, y=y_locs, mode='markers', name='markers',
    marker=dict(
        size=shots_made, sizemode='area', sizeref=2. * max(shots_made) / (11. ** 2), sizemin=2.5,
        color=shot_accuracy, colorscale=colorscale,
        colorbar=dict(
            thickness=15,
            x=0.84,
            y=0.87,
            yanchor='middle',
            len=0.2,
            title=dict(
                text="<B>Accuracy</B>",
                font=dict(
                    size=11,
                    color='#4d4d4d'
                ),
            ),
            tickvals=[marker_cmin, (marker_cmin + marker_cmax) / 2, marker_cmax],
            ticktext=ticktexts,
            tickfont=dict(
                size=11,
                color='#4d4d4d'
            )
        ),
        cmin=marker_cmin, cmax=marker_cmax,
        line=dict(width=1, color='#333333'), symbol='square',
    ),
))
fig.show(config=dict(displayModeBar=False))

### Filter out coordinates with <2 shots taken and improve tooltip

In [140]:
hexbin_stats = pd.DataFrame({'x_locs':x_locs, 'y_locs':y_locs, 'shots_taken':shots_taken, 'shot_accuracy':shot_accuracy})
hexbin_stats = hexbin_stats.loc[hexbin_stats['shots_taken'] > 2].reset_index()

shot_accuracy_filt = hexbin_stats.shot_accuracy
freq_by_hex_filt = hexbin_stats.shots_taken
x_locs_filt = hexbin_stats.x_locs
y_locs_filt = hexbin_stats.y_locs

hexbin_text = [
        '<i>Accuracy: </i>' + str(round(shot_accuracy_filt[i]*100, 1)) + '%<BR>'
        '<i>Frequency: </i>' + str(freq_by_hex_filt[i])
        for i in range(len(freq_by_hex_filt))
]

fig = go.Figure()
draw_plotly_court(fig)
fig.add_trace(go.Scatter(
    x=x_locs_filt, y=y_locs_filt, mode='markers', name='markers',
    marker=dict(
        size=freq_by_hex_filt, sizemode='area', sizeref=2. * max(freq_by_hex_filt) / (10. ** 2), sizemin=2.5,
        color=shot_accuracy_filt, colorscale=colorscale,
        colorbar=dict(
            thickness=15,
            x=0.84,
            y=0.87,
            yanchor='middle',
            len=0.2,
            title=dict(
                text="<B>Accuracy</B>",
                font=dict(
                    size=11,
                    color='#4d4d4d'
                ),
            ),
            tickvals=[marker_cmin, (marker_cmin + marker_cmax) / 2, marker_cmax],
            ticktext=ticktexts,
            tickfont=dict(
                size=11,
                color='#4d4d4d'
            )
        ),
        cmin=marker_cmin, cmax=marker_cmax,
        line=dict(width=1, color='#333333'), symbol='square',
    ),
    text=hexbin_text,
    hoverinfo='text'
))
fig.show(config=dict(displayModeBar=False))